# Vietoris-Rips Filtration Demo

This notebook demonstrates persistent homology by:
1. Generating 5 random points in 2D
2. Computing persistent homology using ripser
3. Visualizing the Vietoris-Rips filtration at different radius values

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Polygon
from scipy.spatial.distance import pdist, squareform
from ripser import ripser
from persim import plot_diagrams
from itertools import combinations

## Generate Random Points

In [ ]:
# Set seed for reproducibility
np.random.seed(42)

# Generate 5 random points in 2D space
points = np.random.rand(5, 2) * 10 - 5  # Scale to [-5, 5] range

print("Generated points:")
for i, p in enumerate(points):
    print(f"Point {i}: ({p[0]:.2f}, {p[1]:.2f})")

# Compute pairwise distances
distances = squareform(pdist(points))
print(f"\nDistance matrix shape: {distances.shape}")

## Compute Persistent Homology

In [ ]:
# Compute persistent homology using Vietoris-Rips filtration
result = ripser(points, maxdim=1)

# Extract persistence diagrams
diagrams = result['dgms']

print("H0 (connected components):")
print(diagrams[0])
print("\nH1 (loops/holes):")
print(diagrams[1])

## Visualize Persistence Diagrams

In [ ]:
# Plot persistence diagrams
plot_diagrams(diagrams)
plt.suptitle('Persistence Diagrams', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Visualize Vietoris-Rips Filtration

Show how the simplicial complex evolves as the radius parameter increases.

In [ ]:
def plot_filtration_step(points, radius, ax, title):
    """
    Plot the Vietoris-Rips complex at a given radius.
    
    Args:
        points: Array of shape (n, 2) containing point coordinates
        radius: The current radius for the filtration
        ax: Matplotlib axis to plot on
        title: Title for the subplot
    """
    n = len(points)
    distances = squareform(pdist(points))
    
    # Find edges (pairs of points within distance r)
    edges = []
    for i in range(n):
        for j in range(i+1, n):
            if distances[i, j] <= 2 * radius:  # Edge exists if balls of radius r overlap
                edges.append((i, j))
    
    # Find triangles (triples where all three pairwise distances ≤ 2*radius)
    triangles = []
    for i, j, k in combinations(range(n), 3):
        if (distances[i, j] <= 2 * radius and 
            distances[i, k] <= 2 * radius and 
            distances[j, k] <= 2 * radius):
            triangles.append((i, j, k))
    
    # Plot triangles (filled)
    for tri in triangles:
        triangle_points = points[list(tri)]
        polygon = Polygon(triangle_points, alpha=0.3, facecolor='lightgray', edgecolor='black', linewidth=1)
        ax.add_patch(polygon)
    
    # Plot edges
    for i, j in edges:
        ax.plot([points[i, 0], points[j, 0]], 
               [points[i, 1], points[j, 1]], 
               'k-', linewidth=1.5, zorder=1)
    
    # Plot points
    ax.scatter(points[:, 0], points[:, 1], c='blue', s=100, zorder=2, edgecolors='black')
    
    # Add labels (optional - comment out if too cluttered)
    labels = ['p0', 'p1', 'p2', 'p3', 'p4']
    for i, (x, y) in enumerate(points):
        ax.text(x, y + 0.3, labels[i], ha='center', fontsize=9)
    
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_aspect('equal')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

In [ ]:
# Choose radius values to visualize
# We'll use a progression similar to the image: 0, small, medium, large values
max_dist = np.max(distances) / 2
radius_values = [0, max_dist * 0.15, max_dist * 0.3, max_dist * 0.5, max_dist * 0.7, max_dist * 0.9]

# Create figure with subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, r in enumerate(radius_values):
    plot_filtration_step(points, r, axes[idx], f'r = {r:.2f}')

plt.tight_layout()
plt.savefig('filtration_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFiltration visualization saved as 'filtration_visualization.png'")

## Interpretation

The visualization shows how the Vietoris-Rips complex evolves:

- **r = 0**: Only individual points (5 connected components, no edges)
- **Small r**: Edges start forming between nearby points
- **Medium r**: More edges appear, possibly forming triangles (filled regions)
- **Large r**: Eventually all points connect into a single component

The persistence diagram captures:
- **H0 (blue)**: When connected components merge (birth = 0, death = when components join)
- **H1 (orange)**: When loops/cycles form and fill in (birth = when loop forms, death = when triangle fills it)